# Prerequisites 

To run this pipeline, you need the `ogc-na` Python module to be installed in your environment (for example, by running `pip install ogc-na`). If you are running this notebook inside the provided Docker container, good news: that has already been take care of for you.

In [ ]:
from ogc.na import ingest_json
import copy

# Running simple uplifts

`uplift_json` can be use to run simple, "low-level" semantic uplifts of JSON content which has already been loaded as a Python `dict` or `list`; the content may come from an API, from a file, or it is being generated by the application running the uplift:

In [ ]:
data = {
    'name': 'Alejandro',
    'age': 38,
    'friends': [
        { 'name': 'Juan', 'age': '-' },
        { 'name': 'María', 'age': '36' },
    ],
    'addresses': [
        {'type': 'home', 'st': '742 Evergreen Terrace' },
        {'type': 'work', 'st': '100 Industrial Way' },
    ],
}

Now we just need the uplift definition. Again, this could come from an API or a file, or be hardcoded as dict, automatically generated...:

In [ ]:
uplift = {
    'transform': [
        # Remove age when empty
        'walk(if type == "object" and .age == "-" then del(.age) else . end)',
    ],
    'types': {
    },
    'context': {
        '$': {
            'foaf': 'http://xmlns.com/foaf/0.1/',
            'xsd': 'http://www.w3.org/2001/XMLSchema#',
            'vcard': 'http://www.w3.org/2006/vcard/ns#',
            'ex': 'https://example.com/vocab#',
            'name': 'foaf:name',
            'friends': 'foaf:knows',
            'age': {
                '@id': 'http://dbpedia.org/ontology/age',
                '@type': 'xsd:integer',
            },
            'addresses': {
                '@id': 'vcard:hasAddress',
                '@context': {
                    'type': '@type',
                    'st': 'vcard:hasStreetAddress',
                    'home': 'ex:HomeAddress',
                    'work': 'ex:WorkAddress',
                },
            },
        }
    }
}

There are two methods that we can use to perform the uplift:

* `uplift_json` will take the data and the uplift definition and transform the former according to the latter, returning a JSON or JSON-LD object, usually as a Python `dict`.
* `generate_graph` goes one step further: it can work with a single uplift definition or with a list of them, and not only it transforms the data (by calling `uplift_json`), but it will also parse it and return an RDFLib `Graph`.

First, let us see what `uplift_json` does:

In [ ]:
# Make a deep copy of the dict, since uplift_json modifies it
data2 = copy.deepcopy(data)
result = ingest_json.uplift_json(data2, uplift)
display(result)

So, basically, the original data is returned, after applying the `transforms`, embedding the `types` and adding the `context`.

If our end goal is to have a transformed (potentially JSON-LD) version of the original data, `uplift_json` can be enough. Let us see what we can do with `generate_graph`:

In [ ]:
# Again, deep copy of the source data
data3 = copy.deepcopy(data)

result = ingest_json.generate_graph(data3, uplift)

print("== Uplifted json (same as ingest_json) ==")
display(result.uplifted_json)
print("\n== RDF version ==")
print(result.graph.serialize())

# Using transform arguments

Uplift definitions can use transform arguments that can be provided at runtime (i.e., when executing `generate_graph` or 'uplift_json'), and that will be available as JQ variables:

In [ ]:
# Clone the data
data4 = copy.deepcopy(data)

# Copy the uplift definition, adding a transform
uplift2 = copy.deepcopy(uplift)
uplift2['transform'].append('."@id" = $data_id')

# Define the arguments
args = { 'data_id': 'https://example.net/data-source-api/1234' }

result = ingest_json.generate_graph(data4, uplift2, transform_args = args)

print(result.graph.serialize())